# Advanced pyALF part 1: Running ALF

In [1]:
from pprint import pprint  # Pretty print
from py_alf import ALF_source, Simulation  # Interface with ALF

## ALF_source
Create instance of `ALF_source`, choosing to checkout the git branch `'211-add-nematic-dirac-hamiltonian'`, since the Nematic Dirac Hamiltonian is not on master.

Reminder: Directory containing the ALF code is taken from environment variable `$ALF_DIR`, if present.

In [2]:
alf_src = ALF_source(
    branch='211-add-nematic-dirac-hamiltonian',
)

Checking out branch 211-add-nematic-dirac-hamiltonian


Check available Hamiltonians:

In [3]:
alf_src.get_ham_names()

['Kondo',
 'Hubbard',
 'Hubbard_Plain_Vanilla',
 'tV',
 'LRC',
 'Z2_Matter',
 'Nematic_Dirac']

Print valid parameters in their defaults for Nematic Dirac Hamiltonian, including QMC parameters:

In [4]:
pprint(alf_src.get_default_params('Nematic_Dirac'))

OrderedDict([('VAR_Nematic_Dirac',
              {'Global_J': {'comment': 'J for proposing global updates',
                            'defined_in_base': False,
                            'value': 1.0},
               'Global_h': {'comment': 'h for proposing global updates',
                            'defined_in_base': False,
                            'value': 3.0},
               'Global_type': {'comment': 'Type of global update. Possible '
                                          "values: 'Wolff', 'Geo', 'switch', "
                                          "'flip'",
                               'defined_in_base': False,
                               'value': ''},
               'Ham_J': {'comment': 'Ferromagnetic Ising interaction',
                         'defined_in_base': False,
                         'value': 1.0},
               'Ham_chem': {'comment': 'Chemical potential',
                            'defined_in_base': False,
                            'value': 0

## Run a series of simulations
Create a list of `Simulation` instances with varying transverse Ising field $h$.

MPI parallelization is enabled, with number of processes set to 4. This results in 4 quasi-independent runs with the same parameters and therefore more precise outcomes.

In [5]:
L = 4
simsL4 = [
    Simulation(
        alf_src,
        'Nematic_Dirac',
        {
            # Model specific parameters
            'Model_vers': 1,
            'L1': L,
            'L2': L,
            'beta': L*4.,
            'Ham_xi': 0.25,
            'Ham_h': h,
            # QMC parameters
            'Ltau': 1,
            'CPU_MAX': .1,
            'NSweep': 20,
        },
        machine='GNU',
        mpi=True,
        n_mpi=4,
        mpiexec='orterun',
    )
    for h in [2.5, 3.0, 3.5, 4.0]]

In [6]:
simsL4

In [7]:
# Simple example for Pythons list comprehension (and f-strings)
[f'h={h}' for h in [2.5, 3.0, 3.5, 4.0]]

['h=2.5', 'h=3.0', 'h=3.5', 'h=4.0']

`Simulation` generates the directory name the simulation is run in out of the non-default parameters, prepended by "ALF_data". For the first simulation, this is:

In [8]:
for sim in simsL4:
    print(sim.get_directories())

['/home/stafusa/ALF/pyALF/Notebooks/ALF_data/Nematic_Dirac_Model_vers=1_L1=4_L2=4_beta=16.0_xi=0.25_h=2.5']
['/home/stafusa/ALF/pyALF/Notebooks/ALF_data/Nematic_Dirac_Model_vers=1_L1=4_L2=4_beta=16.0_xi=0.25_h=3.0']
['/home/stafusa/ALF/pyALF/Notebooks/ALF_data/Nematic_Dirac_Model_vers=1_L1=4_L2=4_beta=16.0_xi=0.25_h=3.5']
['/home/stafusa/ALF/pyALF/Notebooks/ALF_data/Nematic_Dirac_Model_vers=1_L1=4_L2=4_beta=16.0_xi=0.25_h=4.0']


This behaviour can be overwritten through the optional arguments `sim_dir` and `sim_root`.

Compile ALF.

In [9]:
simsL4[0].compile()

Compiling ALF... 
Done.


Run the list of simulations in sequence.

In [10]:
for sim in simsL4:
    sim.run()

Prepare directory "/home/stafusa/ALF/pyALF/Notebooks/ALF_data/Nematic_Dirac_Model_vers=1_L1=4_L2=4_beta=16.0_xi=0.25_h=2.5" for Monte Carlo run.
Resuming previous run.
Run /home/stafusa/ALF/pyALF/Notebooks/ALF/Prog/ALF.out
Prepare directory "/home/stafusa/ALF/pyALF/Notebooks/ALF_data/Nematic_Dirac_Model_vers=1_L1=4_L2=4_beta=16.0_xi=0.25_h=3.0" for Monte Carlo run.
Create new directory.
Run /home/stafusa/ALF/pyALF/Notebooks/ALF/Prog/ALF.out
Prepare directory "/home/stafusa/ALF/pyALF/Notebooks/ALF_data/Nematic_Dirac_Model_vers=1_L1=4_L2=4_beta=16.0_xi=0.25_h=3.5" for Monte Carlo run.
Create new directory.
Run /home/stafusa/ALF/pyALF/Notebooks/ALF/Prog/ALF.out
Prepare directory "/home/stafusa/ALF/pyALF/Notebooks/ALF_data/Nematic_Dirac_Model_vers=1_L1=4_L2=4_beta=16.0_xi=0.25_h=4.0" for Monte Carlo run.
Create new directory.
Run /home/stafusa/ALF/pyALF/Notebooks/ALF/Prog/ALF.out


Check info files produced by ALF. It is strongly advised to take a look at these after finished runs, in particular "Precision Green" and "Precision Phase". These should be small, of order $10^{-8}$ or smaller. If they're bigger, one should decrease the stabilization invervall `Nwrap` (see parameter list `'VAR_QMC'` above). In our case, they're of order $10^{-14}$ and one might consider increasing `Nwrap` to speed up the simulation.

In [11]:
for sim in simsL4:
    sim.print_info_file()

===== /home/stafusa/ALF/pyALF/Notebooks/ALF_data/Nematic_Dirac_Model_vers=1_L1=4_L2=4_beta=16.0_xi=0.25_h=2.5/info =====
 Model is            : Nematic_Dirac           1
 Global_type         :                                                                 
 L1                  :            4
 L2                  :            4
 N_SUN               :            2
 ham_t               :    1.0000000000000000     
 dtau                :   0.10000000000000001     
 beta                :    16.000000000000000     
 Ham_h               :    2.5000000000000000     
 Ham_J               :    1.0000000000000000     
 Ham_xi              :   0.25000000000000000     
 Ham_xi2             :    0.0000000000000000     
 Ham_chem            :    0.0000000000000000     
 Global_J            :    1.0000000000000000     
 Global_h            :    3.0000000000000000     
 Phi_1               :    0.0000000000000000     
 Phi_2               :    0.0000000000000000     
 init_type           : random     

## Parallel Tempering
The simulation object below shows an example of Parallel Tempering, which runs simulations with different parameters in parallel and exchanges configurations between them. The method has been developed to address ergodicity issues, but in this case it's only used to execute a parameter sweep "in one go". For more techincal details on Parallel Tempering, refer to the ALF documentation.

Parallel Tempering is enabled by making `sim_dict` a list of dictionaries, instead of a single dictionary.

In [15]:
L = 6
simL6 = Simulation(
    alf_src,
    'Nematic_Dirac',
    [{
        # Model specific parameters
        'Model_vers': 1,
        'L1': L,
        'L2': L,
        'beta': L*4.,
        'Ham_xi': 0.25,
        'Ham_h': h,
        # QMC parameters
        'Ltau': 1,
        'CPU_MAX': 1.,
        'NSweep': 20,
        'mpi_per_parameter_set': 1,
        'Tempering_calc_det': False,
    } for h in [2.5, 3.0, 3.5, 4.0]],
    machine='GNU',
    mpi=True,
    n_mpi=4,
    mpiexec='orterun',
)

In [16]:
simL6.get_directories()

['/home/stafusa/ALF/pyALF/Notebooks/ALF_data/temper_Nematic_Dirac_Model_vers=1_L1=6_L2=6_beta=24.0_xi=0.25_h=2.5/Temp_0',
 '/home/stafusa/ALF/pyALF/Notebooks/ALF_data/temper_Nematic_Dirac_Model_vers=1_L1=6_L2=6_beta=24.0_xi=0.25_h=2.5/Temp_1',
 '/home/stafusa/ALF/pyALF/Notebooks/ALF_data/temper_Nematic_Dirac_Model_vers=1_L1=6_L2=6_beta=24.0_xi=0.25_h=2.5/Temp_2',
 '/home/stafusa/ALF/pyALF/Notebooks/ALF_data/temper_Nematic_Dirac_Model_vers=1_L1=6_L2=6_beta=24.0_xi=0.25_h=2.5/Temp_3']

In [17]:
simL6.run()

Prepare directory "/home/stafusa/ALF/pyALF/Notebooks/ALF_data/temper_Nematic_Dirac_Model_vers=1_L1=6_L2=6_beta=24.0_xi=0.25_h=2.5" for Monte Carlo run.
Prepare directory "/home/stafusa/ALF/pyALF/Notebooks/ALF_data/temper_Nematic_Dirac_Model_vers=1_L1=6_L2=6_beta=24.0_xi=0.25_h=2.5/Temp_0" for Monte Carlo run.
Prepare directory "/home/stafusa/ALF/pyALF/Notebooks/ALF_data/temper_Nematic_Dirac_Model_vers=1_L1=6_L2=6_beta=24.0_xi=0.25_h=2.5/Temp_1" for Monte Carlo run.
Prepare directory "/home/stafusa/ALF/pyALF/Notebooks/ALF_data/temper_Nematic_Dirac_Model_vers=1_L1=6_L2=6_beta=24.0_xi=0.25_h=2.5/Temp_2" for Monte Carlo run.
Prepare directory "/home/stafusa/ALF/pyALF/Notebooks/ALF_data/temper_Nematic_Dirac_Model_vers=1_L1=6_L2=6_beta=24.0_xi=0.25_h=2.5/Temp_3" for Monte Carlo run.
Run /home/stafusa/ALF/pyALF/Notebooks/ALF/Prog/ALF.out


In [18]:
simL6.print_info_file()

/home/stafusa/ALF/pyALF/Notebooks/ALF_data/temper_Nematic_Dirac_Model_vers=1_L1=6_L2=6_beta=24.0_xi=0.25_h=2.5/Temp_0/info does not exist.


In [23]:
L = 8
simL8 = Simulation(
    alf_src,
    'Nematic_Dirac',
    [{
        # Model specific parameters
        'Model_vers': 1,
        'L1': L,
        'L2': L,
        'beta': L*4.,
        'Ham_xi': 0.25,
        'Ham_h': h,
        # QMC parameters
        'Ltau': 1,
        'CPU_MAX': 7.,
        'NSweep': 20,
        'Nbin': 1,
        'mpi_per_parameter_set': 1,
        'Tempering_calc_det': False,
    } for h in [2.5, 3.0, 3.5, 4.0]],
    machine='GNU',
    mpi=True,
    n_mpi=4,
    mpiexec='orterun',
)

In [ ]:
simL8.run()

Prepare directory "/home/stafusa/ALF/pyALF/Notebooks/ALF_data/temper_Nematic_Dirac_Model_vers=1_L1=8_L2=8_beta=32.0_xi=0.25_h=2.5" for Monte Carlo run.
Prepare directory "/home/stafusa/ALF/pyALF/Notebooks/ALF_data/temper_Nematic_Dirac_Model_vers=1_L1=8_L2=8_beta=32.0_xi=0.25_h=2.5/Temp_0" for Monte Carlo run.
Prepare directory "/home/stafusa/ALF/pyALF/Notebooks/ALF_data/temper_Nematic_Dirac_Model_vers=1_L1=8_L2=8_beta=32.0_xi=0.25_h=2.5/Temp_1" for Monte Carlo run.
Prepare directory "/home/stafusa/ALF/pyALF/Notebooks/ALF_data/temper_Nematic_Dirac_Model_vers=1_L1=8_L2=8_beta=32.0_xi=0.25_h=2.5/Temp_2" for Monte Carlo run.
Prepare directory "/home/stafusa/ALF/pyALF/Notebooks/ALF_data/temper_Nematic_Dirac_Model_vers=1_L1=8_L2=8_beta=32.0_xi=0.25_h=2.5/Temp_3" for Monte Carlo run.
Run /home/stafusa/ALF/pyALF/Notebooks/ALF/Prog/ALF.out


In [ ]:
simL8.print_info_file()

In [ ]:
L = 10
simL10 = Simulation(
        alf_src,
        'Nematic_Dirac',
        [{
            # Model specific parameters
            'Model_vers': 1,
            'L1': L,
            'L2': L,
            'beta': L*4.,
            'Ham_xi': 0.25,
            'Ham_h': h,
            # QMC parameters
            'Ltau': 1,
            'CPU_MAX': 18,
            'NSweep': 20,
            'mpi_per_parameter_set': 1,
        } for h in [2.5, 3.0, 3.5, 4.0]],
        machine='GNU',
        mpi=True,
        n_mpi=4,
        mpiexec='orterun',
    )

In [ ]:
simL10.run()

In [ ]:
simL10.print_info_file()

In [ ]:
## Only preparing runs

In many cases, it might not be feasible to execute ALF directly through pyALF, for example when using an HPC scheduler, but one might still like to use pyALF for preparing the simulation directories. In this case the two options `copy_bin` and `only_prep` of {func}`py_alf.Simulation.run` come in handy. This will also serve as a demonstration of the keyword arguments `sim_root` and `sim_dir`.

In [ ]:
L = 20
simsL20 = [
    Simulation(
        alf_src,
        'Nematic_Dirac',
        {
            # Model specific parameters
            'Model_vers': 1,
            'L1': L,
            'L2': L,
            'beta': L*4.,
            'Ham_xi': 0.25,
            'Ham_h': h,
            # QMC parameters
            'Ltau': 1,
            'CPU_MAX': 48,
            'NSweep': 20,
        },
        machine='GNU',
        mpiexec='orterun',
        mpi=True,
        n_mpi=4,
        sim_root="~/NematicDiracL20",
        sim_dir=f"h{h:2.2f}",
    )
    for h in [3.1, 3.2, 3.3, 3.4, 3.5]]

In [ ]:
simsL20.compile()

In [ ]:
for sim in simsL20:
    sim.run(copy_bin=True, only_prep=True)

In [ ]:
ls NematicDiracL20/*